### FHVHV Data Pipeline - Stage 3: Demand Forecasting

**Pipeline Position:** Stage 3 of 4

- Stage 0: Data Download (Complete)
- Stage 1: Data Validation (Complete)
- Stage 2: Exploratory Analysis (Complete)
- Stage 3: Modeling ← THIS NOTEBOOK

**Objective:** Build and evaluate forecasting models to predict daily zone demand.

**Models:** Baseline (Seasonal Naive), Prophet, XGBoost

**Approach:** Develop on single zone → Evaluate → Scale to 100 zones

**EDA Insights Applied:**
- Weekend-dominant demand (17% higher) → models must capture weekly seasonality
- Moderate yearly seasonality (13.7%) → Prophet will handle this
- Low variability (CV < 0.3) → simpler models may perform well
- 40% zone pairs highly correlated → potential for shared parameters

---

#### 1. Setup

Load libraries, data, and define evaluation metrics for model comparison.

##### 1.1 Import Libraries

In [6]:
# Core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuration
import warnings
warnings.filterwarnings('ignore')

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 4),
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8
})

print("Libraries imported successfully")

Libraries imported successfully


##### 1.2 Load Data

In [ ]:
# What: Load zone-daily aggregated data from EDA stage
# Why: This is our modeling-ready dataset with time features

DATA_DIR = Path("../data/processed")

zone_daily = pd.read_parquet(DATA_DIR / "zone_daily.parquet")
selected_zones = pd.read_csv(DATA_DIR / "zone_metadata.csv")

print("DATA LOADED")
print("=" * 40)
print(f"Zone-daily records: {len(zone_daily):,}")
print(f"Zones: {zone_daily['zone_id'].nunique()}")
print(f"Date range: {zone_daily['date'].min()} to {zone_daily['date'].max()}")
print(f"Columns: {len(zone_daily.columns)}")

DATA LOADED
Zone-daily records: 109,600
Zones: 100
Date range: 2022-01-01 00:00:00 to 2024-12-31 00:00:00
Columns: 28


##### 1.3 Define Evaluation Metrics

In [8]:
# What: Create evaluation function for consistent model comparison
# Why: Need standard metrics across all models (MAE, RMSE, MAPE)

def evaluate_forecast(actual, predicted, model_name="Model"):
    """Calculate forecast accuracy metrics."""
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return {
        'Model': model_name,
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2),
        'MAPE': round(mape, 2)
    }

print("Evaluation function defined")
print("Metrics: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error), MAPE (Mean Absolute Percentage Error)")

Evaluation function defined
Metrics: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error), MAPE (Mean Absolute Percentage Error)


---

#### 2. Prepare Train/Test Split

Create time-based split and select pilot zone for model development.

##### 2.1 Create Time-Based Split

In [12]:
# What: Split data by time - train on historical, test on recent
# Why: Time series must respect temporal order (no data leakage)

TRAIN_END = '2024-06-30'
TEST_START = '2024-07-01'

train = zone_daily[zone_daily['date'] <= TRAIN_END].copy()
test = zone_daily[zone_daily['date'] >= TEST_START].copy()

print("TRAIN/TEST SPLIT")
print("=" * 40)
print(f"Train: {train['date'].min()} to {train['date'].max()} ({len(train):,} rows)")
print(f"Test:  {test['date'].min()} to {test['date'].max()} ({len(test):,} rows)")
print(f"\nTrain months: {train['date'].dt.to_period('M').nunique()}")
print(f"Test months: {test['date'].dt.to_period('M').nunique()}")

TRAIN/TEST SPLIT
Train: 2022-01-01 00:00:00 to 2024-06-30 00:00:00 (91,200 rows)
Test:  2024-07-01 00:00:00 to 2024-12-31 00:00:00 (18,400 rows)

Train months: 30
Test months: 6


##### 2.2 Select Pilot Zone

In [15]:
# Check what's in the data
print("zone_daily columns:", zone_daily.columns.tolist())
print("zone_daily zone_id dtype:", zone_daily['zone_id'].dtype)
print("zone_daily zone_id sample:", zone_daily['zone_id'].head())
print("Zone 138 exists?", 138 in zone_daily['zone_id'].values)
print("zone_daily date dtype:", zone_daily['date'].dtype)
print("zone_daily date sample:", zone_daily['date'].head())

zone_daily columns: ['date', 'zone_id', 'daily_trips', 'daily_total_minutes', 'daily_avg_minutes', 'total_trip_miles', 'avg_trip_miles', 'year', 'month', 'day_of_week', 'day_name', 'is_weekend', 'month_name', 'is_outlier', 'day', 'day_of_year', 'week_of_year', 'is_month_start', 'is_month_end', 'season', 'days_since_start', 'lag_1', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7', 'is_holiday']
zone_daily zone_id dtype: object
zone_daily zone_id sample: 0    100
1    100
2    100
3    100
4    100
Name: zone_id, dtype: object
Zone 138 exists? False
zone_daily date dtype: datetime64[us]
zone_daily date sample: 0   2022-01-01
1   2022-01-02
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: date, dtype: datetime64[us]


In [14]:
# What: Select top zone by volume for model development
# Why: Develop and tune on one zone before scaling to all 100

pilot_zone_id = int(selected_zones.iloc[0]['zone_id'])

pilot_train = train[train['zone_id'] == pilot_zone_id].copy()
pilot_test = test[test['zone_id'] == pilot_zone_id].copy()

print(f"PILOT ZONE: {pilot_zone_id}")
print("=" * 40)
print(f"Train records: {len(pilot_train)}")
print(f"Test records: {len(pilot_test)}")
print(f"Avg daily demand: {pilot_train['daily_trips'].mean():,.0f} trips")

PILOT ZONE: 138
Train records: 0
Test records: 0
Avg daily demand: nan trips


In [13]:
# What: Select top zone by volume for model development
# Why: Develop and tune on one zone before scaling to all 100

pilot_zone_id = int(selected_zones.iloc[0]['zone_id'])

pilot_train = train[train['zone_id'] == pilot_zone_id].copy()
pilot_test = test[test['zone_id'] == pilot_zone_id].copy()

print(f"PILOT ZONE: {pilot_zone_id}")
print("=" * 40)
print(f"Train records: {len(pilot_train)}")
print(f"Test records: {len(pilot_test)}")
print(f"Avg daily demand: {pilot_train['daily_trips'].mean():,.0f} trips")

PILOT ZONE: 138
Train records: 0
Test records: 0
Avg daily demand: nan trips


---

#### 3. Build Baseline Model

Implement seasonal naive baseline (7-day lag) to set performance benchmark.

##### 3.1 Implement Seasonal Naive

In [ ]:
# What: Predict using value from 7 days ago (same day of week)
# Why: Simple benchmark that captures weekly seasonality

# Create lag-7 forecast for test period
pilot_test['baseline_pred'] = pilot_test['daily_trips'].shift(7)

# For first 7 days of test, use last 7 days of train
last_week_train = pilot_train.tail(7)['daily_trips'].values
pilot_test.iloc[:7, pilot_test.columns.get_loc('baseline_pred')] = last_week_train

print("Baseline model: Seasonal Naive (7-day lag)")
print(f"Logic: forecast[t] = actual[t-7]")

##### 3.2 Evaluate Baseline

In [ ]:
# What: Calculate baseline metrics
# Why: This sets the bar - other models must beat this

baseline_results = evaluate_forecast(
    pilot_test['daily_trips'], 
    pilot_test['baseline_pred'],
    model_name="Baseline (Seasonal Naive)"
)

print("BASELINE PERFORMANCE")
print("=" * 40)
print(f"MAE:  {baseline_results['MAE']:,.0f} trips/day")
print(f"RMSE: {baseline_results['RMSE']:,.0f} trips/day")
print(f"MAPE: {baseline_results['MAPE']:.1f}%")

##### 3.3 Visualize Baseline Predictions

In [ ]:
# What: Plot baseline forecast vs actual demand
# Why: Visual check of baseline performance

fig, ax = plt.subplots(figsize=(14, 4))

ax.plot(pilot_test['date'], pilot_test['daily_trips'], 
        label='Actual', color='steelblue', linewidth=1.5)
ax.plot(pilot_test['date'], pilot_test['baseline_pred'], 
        label='Baseline (7-day lag)', color='coral', linewidth=1.5, linestyle='--')

ax.set_xlabel('Date')
ax.set_ylabel('Daily Trips')
ax.set_title(f'Baseline Forecast vs Actual - Zone {pilot_zone_id}')
ax.legend()
plt.tight_layout()
plt.show()

**Interpretation:** [Add after running - how well does the simple lag work?]

---

#### 4. Build Prophet Model

Train Prophet to capture trend, weekly/yearly seasonality, and holidays.

##### 4.1 Prepare Data for Prophet

In [ ]:
# What: Format data for Prophet (requires 'ds' and 'y' columns)
# Why: Prophet has specific input format requirements

prophet_train = pilot_train[['date', 'daily_trips']].copy()
prophet_train.columns = ['ds', 'y']

prophet_test = pilot_test[['date', 'daily_trips']].copy()
prophet_test.columns = ['ds', 'y']

print(f"Prophet train shape: {prophet_train.shape}")
print(f"Prophet test shape: {prophet_test.shape}")

##### 4.2 Configure and Train Model

In [ ]:
# What: Initialize Prophet with weekly and yearly seasonality
# Why: EDA showed both weekly (17% weekend lift) and yearly (13.7% range) patterns

model_prophet = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,  # Data is already daily aggregated
    seasonality_mode='multiplicative',  # Better for % changes
    changepoint_prior_scale=0.05  # Controls trend flexibility
)

# Add US holidays (NYC market)
model_prophet.add_country_holidays(country_name='US')

# Train model
model_prophet.fit(prophet_train)

print("Prophet model trained")
print(f"Changepoints detected: {len(model_prophet.changepoints)}")

##### 4.3 Generate Forecast

In [ ]:
# What: Generate predictions for test period
# Why: Need forecasts to compare against actual values

future = prophet_test[['ds']].copy()
prophet_forecast = model_prophet.predict(future)

# Store predictions
pilot_test['prophet_pred'] = prophet_forecast['yhat'].values

print(f"Prophet forecast generated: {len(prophet_forecast)} days")

##### 4.4 Evaluate Prophet

In [ ]:
# What: Calculate Prophet metrics and compare to baseline
# Why: Determine if Prophet adds value over simple baseline

prophet_results = evaluate_forecast(
    pilot_test['daily_trips'], 
    pilot_test['prophet_pred'],
    model_name="Prophet"
)

# Calculate improvement over baseline
improvement = (baseline_results['MAE'] - prophet_results['MAE']) / baseline_results['MAE'] * 100

print("PROPHET PERFORMANCE")
print("=" * 40)
print(f"MAE:  {prophet_results['MAE']:,.0f} trips/day")
print(f"RMSE: {prophet_results['RMSE']:,.0f} trips/day")
print(f"MAPE: {prophet_results['MAPE']:.1f}%")
print(f"\nImprovement over baseline: {improvement:+.1f}%")

##### 4.5 Visualize Components

In [ ]:
# What: Plot Prophet's decomposition (trend, weekly, yearly)
# Why: Understand what patterns Prophet learned

fig = model_prophet.plot_components(prophet_forecast)
plt.suptitle(f'Prophet Components - Zone {pilot_zone_id}', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

**Interpretation:** [Add after running - what patterns did Prophet capture?]

---

#### 5. Build XGBoost Model

Train XGBoost using engineered time features and lag variables.

##### 5.1 Create Lag Features

In [ ]:
# What: Create lag features for XGBoost
# Why: XGBoost needs explicit features - doesn't learn time patterns automatically

def create_lag_features(df, target_col='daily_trips', lags=[1, 7, 14, 28]):
    """Create lag features for time series modeling."""
    df = df.copy()
    for lag in lags:
        df[f'lag_{lag}'] = df.groupby('zone_id')[target_col].shift(lag)
    
    # Rolling features
    df['rolling_mean_7'] = df.groupby('zone_id')[target_col].transform(
        lambda x: x.shift(1).rolling(7, min_periods=1).mean()
    )
    df['rolling_mean_28'] = df.groupby('zone_id')[target_col].transform(
        lambda x: x.shift(1).rolling(28, min_periods=1).mean()
    )
    return df

# Apply to full dataset then re-split
zone_daily_features = create_lag_features(zone_daily)

# Re-split with new features
train_xgb = zone_daily_features[zone_daily_features['date'] <= TRAIN_END].copy()
test_xgb = zone_daily_features[zone_daily_features['date'] >= TEST_START].copy()

# Filter to pilot zone
pilot_train_xgb = train_xgb[train_xgb['zone_id'] == pilot_zone_id].copy()
pilot_test_xgb = test_xgb[test_xgb['zone_id'] == pilot_zone_id].copy()

print(f"Lag features created: lag_1, lag_7, lag_14, lag_28, rolling_mean_7, rolling_mean_28")

##### 5.2 Prepare Features and Target

In [ ]:
# What: Select features for XGBoost model
# Why: Define which columns are predictors vs target

feature_cols = [
    # Time features (from EDA)
    'month', 'day_of_week', 'is_weekend', 'is_holiday',
    # Lag features
    'lag_1', 'lag_7', 'lag_14', 'lag_28',
    # Rolling features
    'rolling_mean_7', 'rolling_mean_28'
]

target_col = 'daily_trips'

# Remove rows with NaN from lag creation
pilot_train_xgb = pilot_train_xgb.dropna(subset=feature_cols)
pilot_test_xgb = pilot_test_xgb.dropna(subset=feature_cols)

X_train = pilot_train_xgb[feature_cols]
y_train = pilot_train_xgb[target_col]

X_test = pilot_test_xgb[feature_cols]
y_test = pilot_test_xgb[target_col]

print(f"Features: {len(feature_cols)}")
print(f"Train samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

##### 5.3 Train Model

In [ ]:
# What: Train XGBoost regressor
# Why: Tree-based model that handles non-linear relationships

model_xgb = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

model_xgb.fit(X_train, y_train)

# Generate predictions
xgb_pred = model_xgb.predict(X_test)

print("XGBoost model trained")

##### 5.4 Evaluate XGBoost

In [ ]:
# What: Calculate XGBoost metrics and compare to baseline
# Why: Determine if XGBoost adds value

xgb_results = evaluate_forecast(
    y_test, 
    xgb_pred,
    model_name="XGBoost"
)

# Calculate improvement over baseline
improvement_xgb = (baseline_results['MAE'] - xgb_results['MAE']) / baseline_results['MAE'] * 100

print("XGBOOST PERFORMANCE")
print("=" * 40)
print(f"MAE:  {xgb_results['MAE']:,.0f} trips/day")
print(f"RMSE: {xgb_results['RMSE']:,.0f} trips/day")
print(f"MAPE: {xgb_results['MAPE']:.1f}%")
print(f"\nImprovement over baseline: {improvement_xgb:+.1f}%")

##### 5.5 Feature Importance

In [ ]:
# What: Display feature importance from XGBoost
# Why: Understand which features drive predictions

importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': model_xgb.feature_importances_
}).sort_values('importance', ascending=True)

fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(importance_df['feature'], importance_df['importance'], color='steelblue')
ax.set_xlabel('Feature Importance')
ax.set_title(f'XGBoost Feature Importance - Zone {pilot_zone_id}')
plt.tight_layout()
plt.show()

**Interpretation:** [Add after running - which features matter most?]

---

#### 6. Compare Models (Single Zone)

Compare all models and select best approach for scaling.

##### 6.1 Metrics Comparison Table

In [ ]:
# What: Compare all model metrics in one table
# Why: Easy side-by-side comparison for model selection

results_df = pd.DataFrame([baseline_results, prophet_results, xgb_results])
results_df['vs_Baseline'] = ((results_df['MAE'] - baseline_results['MAE']) / baseline_results['MAE'] * 100).round(1)
results_df['vs_Baseline'] = results_df['vs_Baseline'].apply(lambda x: f"{x:+.1f}%")

print("MODEL COMPARISON")
print("=" * 60)
display(results_df)

##### 6.2 Visualize Forecasts vs Actual

In [ ]:
# What: Plot all forecasts against actual values
# Why: Visual comparison of model performance

fig, ax = plt.subplots(figsize=(14, 5))

# Actual
ax.plot(pilot_test['date'], pilot_test['daily_trips'], 
        label='Actual', color='black', linewidth=2)

# Baseline
ax.plot(pilot_test['date'], pilot_test['baseline_pred'], 
        label='Baseline', color='gray', linewidth=1, linestyle='--', alpha=0.7)

# Prophet
ax.plot(pilot_test['date'], pilot_test['prophet_pred'], 
        label='Prophet', color='coral', linewidth=1.5)

# XGBoost
ax.plot(pilot_test_xgb['date'], xgb_pred, 
        label='XGBoost', color='steelblue', linewidth=1.5)

ax.set_xlabel('Date')
ax.set_ylabel('Daily Trips')
ax.set_title(f'Model Comparison - Zone {pilot_zone_id}')
ax.legend()
plt.tight_layout()
plt.show()

##### 6.3 Select Best Approach

In [ ]:
# What: Determine best model for scaling
# Why: Scale only the best performing approach to all zones

best_model = results_df.loc[results_df['MAE'].idxmin(), 'Model']
best_mae = results_df['MAE'].min()

print("BEST MODEL SELECTION")
print("=" * 40)
print(f"Best model: {best_model}")
print(f"Best MAE: {best_mae:,.0f} trips/day")

**Interpretation:** [Add after running - which model wins and why?]

---

#### 7. Scale to All Zones

Apply best model(s) to all 100 zones and aggregate results.

##### 7.1 Build Forecasting Pipeline

In [ ]:
# What: Create function to train and forecast for any zone
# Why: Reusable pipeline for scaling to 100 zones

def forecast_zone_prophet(zone_id, train_df, test_df):
    """Train Prophet and generate forecasts for a single zone."""
    # Filter to zone
    zone_train = train_df[train_df['zone_id'] == zone_id][['date', 'daily_trips']].copy()
    zone_test = test_df[test_df['zone_id'] == zone_id][['date', 'daily_trips']].copy()
    
    # Format for Prophet
    zone_train.columns = ['ds', 'y']
    zone_test.columns = ['ds', 'y']
    
    # Train
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )
    model.add_country_holidays(country_name='US')
    model.fit(zone_train)
    
    # Forecast
    forecast = model.predict(zone_test[['ds']])
    
    # Evaluate
    mae = mean_absolute_error(zone_test['y'], forecast['yhat'])
    
    return {
        'zone_id': zone_id,
        'mae': mae,
        'predictions': forecast['yhat'].values
    }

print("Forecasting pipeline defined")

##### 7.2 Run on All Zones

In [ ]:
# What: Run forecasting pipeline on all 100 zones
# Why: Generate forecasts for entire dataset

from tqdm import tqdm

zone_ids = zone_daily['zone_id'].unique().tolist()
all_results = []

print(f"Forecasting {len(zone_ids)} zones...")

for zone_id in tqdm(zone_ids):
    try:
        result = forecast_zone_prophet(zone_id, train, test)
        all_results.append(result)
    except Exception as e:
        print(f"Zone {zone_id} failed: {e}")

print(f"\nCompleted: {len(all_results)} zones")

##### 7.3 Aggregate Results

In [ ]:
# What: Summarize performance across all zones
# Why: Understand overall model effectiveness

zone_results_df = pd.DataFrame([{'zone_id': r['zone_id'], 'MAE': r['mae']} for r in all_results])

print("ALL-ZONE PERFORMANCE SUMMARY")
print("=" * 40)
print(f"Zones forecasted: {len(zone_results_df)}")
print(f"\nMAE Statistics:")
print(f"  Mean:   {zone_results_df['MAE'].mean():,.0f} trips/day")
print(f"  Median: {zone_results_df['MAE'].median():,.0f} trips/day")
print(f"  Min:    {zone_results_df['MAE'].min():,.0f} trips/day")
print(f"  Max:    {zone_results_df['MAE'].max():,.0f} trips/day")

---

#### 8. Evaluate Overall Performance

Analyze model performance across zones to identify patterns and problem areas.

##### 8.1 Performance by Zone

In [ ]:
# What: Visualize MAE distribution across zones
# Why: Identify if performance varies by zone

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Histogram
ax1.hist(zone_results_df['MAE'], bins=20, color='steelblue', edgecolor='white')
ax1.axvline(zone_results_df['MAE'].median(), color='coral', linestyle='--', label='Median')
ax1.set_xlabel('MAE (trips/day)')
ax1.set_ylabel('Number of Zones')
ax1.set_title('MAE Distribution Across Zones')
ax1.legend()

# Sorted bar
sorted_results = zone_results_df.sort_values('MAE', ascending=False)
ax2.bar(range(len(sorted_results)), sorted_results['MAE'], color='steelblue')
ax2.set_xlabel('Zone (sorted by MAE)')
ax2.set_ylabel('MAE (trips/day)')
ax2.set_title('MAE by Zone (Worst to Best)')

plt.tight_layout()
plt.show()

##### 8.2 Identify Problem Zones

In [ ]:
# What: Find zones with highest forecast error
# Why: Identify zones that may need different approach

print("TOP 10 WORST PERFORMING ZONES")
print("=" * 40)
worst_zones = zone_results_df.nlargest(10, 'MAE')
display(worst_zones)

print("\nTOP 10 BEST PERFORMING ZONES")
print("=" * 40)
best_zones = zone_results_df.nsmallest(10, 'MAE')
display(best_zones)

**Interpretation:** [Add after running - what patterns emerge? Do high-volume zones perform differently?]

---

#### 9. Save Results

Save forecasts and model artifacts for future use.

##### 9.1 Save Forecasts

In [ ]:
# What: Save zone-level results and forecasts
# Why: Preserve outputs for reporting and analysis

OUTPUT_DIR = Path("../data/model_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# Save zone performance summary
zone_results_df.to_csv(OUTPUT_DIR / "zone_forecast_performance.csv", index=False)
print(f"Saved: {OUTPUT_DIR / 'zone_forecast_performance.csv'}")

# Save comparison results
results_df.to_csv(OUTPUT_DIR / "model_comparison.csv", index=False)
print(f"Saved: {OUTPUT_DIR / 'model_comparison.csv'}")

---

### Conclusion

[Add after running - summarize results, best model, key findings]

### Future Work

- Hyperparameter tuning for Prophet and XGBoost
- Ensemble methods (blend Prophet + XGBoost)
- Zone-specific models for problem zones
- Cross-validation for more robust evaluation
- Feature engineering improvements